In [1]:
import random
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set, Tuple

import networkx as nx
import numpy as np
import pandas as pd

from icm_diffusion import simulate_diffusion_ICM
from src.graph import (
    create_polarized_graph,
    graph_loader,
    random_color_graph,
    spectral_bipartition_coloring,
)
from src.seed import seed_degree, seed_random, seed_mia


In [2]:
from edge_addition import (
    edge_addition_custom,
    edge_addition_adamic_adar,
    edge_addition_preferential_attachment,
    edge_addition_jaccard,
    edge_addition_degree,
    edge_addition_topk,
    edge_addition_prob,
    edge_addition_kkt,
    edge_addition_random,
)

In [3]:
# Function to evaluate and compare the graph modifications
def evaluate_graph_modifications(G, seeds, k, max_iter, budget):
    # Simulate diffusion on the original graph
    count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seeds, max_iter)
    
    # Results for the original graph
    original_results = pd.DataFrame({
        'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
        'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
    })

    # Define a list of modification functions
    modification_functions = {
        'Adamic Adar': edge_addition_adamic_adar,
        'PrefAtt': edge_addition_preferential_attachment,
        'Jaccard': edge_addition_jaccard,
        'Degree': edge_addition_degree,
        'TopK': edge_addition_topk,
        'Prob': edge_addition_prob,
        'KKT': edge_addition_kkt,
        'Random': edge_addition_random,
        'Custom': edge_addition_custom
    }

    combined_results = original_results.copy()

    # Evaluate each graph modification
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        count, count_std, color_count, color_count_std = simulate_diffusion_ICM(modified_graph, seeds, max_iter, budget)

        adapted_results = pd.DataFrame({
            'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
            f'Adapted Graph {method_name}': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
        })

        combined_results = pd.merge(combined_results, adapted_results, on='Metric')

    # Get the number of nodes and edges for all graphs
    graph_info = {
        'Metric': ['Number of Nodes', 'Number of Edges'],
        'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    }
    
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        graph_info[f'Adapted Graph {method_name}'] = [modified_graph.number_of_nodes(), modified_graph.number_of_edges()]
    
    graph_info_df = pd.DataFrame(graph_info)

    # Combine all results into one DataFrame
    final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

    # Transpose the DataFrame and set the first row as the header
    final_results = final_results.T
    final_results.columns = final_results.iloc[0]  # Set the first row as the column names
    final_results = final_results.drop(final_results.index[0])  # Drop the first row

    return final_results

In [4]:
#G = create_polarized_graph(1000, 0.2, 0.01)
G = graph_loader('datasets/congress_network/congress.edgelist')
#color the graph
spectral_bipartition_coloring(G)

Number of Nodes: 475
Number of Edges: 13289


/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [5]:
seed = seed_mia(G, 100)
k = 15
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:02<00:00, 477.06it/s]


TypeError: edge_addition_adamic_adar() missing 1 required positional argument: 'budget'

In [ ]:
seed = seed_random(G, 100)
k = 15
max_iter = 1000
final_results = evaluate_graph_modifications(G, seed, k, max_iter)
final_results

100%|██████████| 1000/1000 [00:02<00:00, 455.18it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,475.0,13289.0,223.03,64.781,26.391,18.207
Adapted Graph Adamic Adar,475.0,14208.0,229.272,66.567,28.094,18.541
Adapted Graph PrefAtt,475.0,14789.0,246.119,65.332,48.205,24.652
Adapted Graph Jaccard,475.0,14319.0,240.52,65.397,28.046,17.125
Adapted Graph Degree,475.0,14604.0,221.173,64.224,31.161,23.203
Adapted Graph TopK,475.0,14469.0,223.147,61.021,30.147,21.067
Adapted Graph Prob,475.0,14789.0,248.579,64.549,53.883,25.634
Adapted Graph KKT,475.0,14483.0,220.121,61.846,30.479,23.07
Adapted Graph Random,475.0,14700.0,220.942,64.434,31.995,22.866
Adapted Graph Custom,475.0,14705.0,218.555,64.62,29.696,21.81
